In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import statsmodels.discrete.discrete_model as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

This dataset ...

In [ ]:
heart = pd.read_csv('../input/heart.csv')

In [ ]:
heart.head()

Several of the variables are actually categorical (encoded in integers 1, 2, ...). Let's make dummy variables for them.

In [ ]:
dummyVars = ['cp', 'restecg', 'slope', 'thal']  # We found that these variables specifically are in need of dummy variables

heart = pd.get_dummies(heart, columns=dummyVars)

heart.head()

The variable names appear to be in fortran notation, let's make them better.

In [ ]:
# Checkout https://archive.ics.uci.edu/ml/datasets/heart+Disease for a description of each variable

renameColumns = {'trestbps': 'Resting Blood Pressure (mmHg)',
                 'chol': 'Serum Cholestoral (mg/dl)',
                 'fbs': 'Fasting Blood Sugar > 120 mg/dl',
                 'thalach': 'Maximum Heartrate Achieved',
                 'exang': 'Exercise Induced Angina',
                 'oldpeak': 'ST Depression',
                 'ca': 'Number of Major Blood Vessels',
                 'target': 'Heart Disease Present',
                 'cp_0': 'Chest Pain Typical Angina',
                 'cp_1': 'Chest Pain Atypical Angina',
                 'cp_2': 'Chest Pain Non-Anginal Pain',
                 'cp_3': 'Chest Pain Asymptomatic',
                 'restecg_0': 'Rest ECG normal',
                 'restecg_1': 'Rest ECG ST-T Wave Abnormality',
                 'restecg_2': 'Rest ECG Left Ventricular Hypertrophy',
                 'slope_0': 'Slope Upsloping',
                 'slope_1': 'Slope Flat',
                 'slope_2': 'Slope Downsloping',
                 'thal_0': 'Thalassemia NaN',  # 0 is not a valid category
                 'thal_1': 'Thalasemmia Normal',
                 'thal_2': 'Thalasemmia Fixed Defect',
                 'thal_3': 'Thalasemmia Reversable Defect'}

heart.rename(mapper = renameColumns, axis = 1, inplace=True)
heart.head()

Let's get rid of the NaN column

In [ ]:
print(heart['Thalassemia NaN'].sum())

heart.drop(columns = 'Thalassemia NaN', inplace = True)

print('Thalassemia NaN' in heart.columns)

Now a heatmap of cross correlation

In [ ]:
plt.figure(figsize = (20, 15))
sns.heatmap(heart.corr(), annot = True, fmt = '.1f')
plt.show()

In [ ]:
sns.catplot(x = 'sex', y = 'age', hue = 'Heart Disease Present', data = heart, kind = 'violin', split = True)

In [ ]:
heart.columns

In [ ]:
sns.catplot('Heart Disease Present', col = 'sex', kind = 'count', data = heart)

In [ ]:
nMales = np.sum(heart['sex'] == 1)
nFemales = len(heart) - nMales

print(nMales)
print(nFemales)

In [ ]:
sns.catplot(x = 'Heart Disease Present', y = 'age', kind = 'violin', data = heart)

In [ ]:
sns.catplot(x = 'Heart Disease Present', y = 'Resting Blood Pressure (mmHg)', kind = 'violin', data = heart)

In [ ]:
sns.catplot(x = 'Heart Disease Present', y = 'Serum Cholestoral (mg/dl)', kind = 'violin', data = heart)

In [ ]:

sns.catplot('Heart Disease Present', col = 'Fasting Blood Sugar > 120 mg/dl', data = heart, kind = 'count')

In [ ]:
sns.catplot(x = 'Heart Disease Present', y = 'Maximum Heartrate Achieved', kind = 'violin', data = heart)

In [ ]:
sns.catplot('Heart Disease Present', col = 'Exercise Induced Angina', kind = 'count', data = heart)

In [ ]:
sns.catplot(x = 'Heart Disease Present', y = 'ST Depression', kind = 'violin', data = heart)

In [ ]:
sns.catplot('Heart Disease Present', col = 'Number of Major Blood Vessels', data = heart, kind = 'count')

In [ ]:
heart.groupby('Heart Disease Present').sum()[[column for column in heart.columns if 'Chest Pain' in column]].plot.bar()

In [ ]:
heart.groupby('Heart Disease Present').sum()[[column for column in heart.columns if 'ECG' in column]].plot.bar()

In [ ]:
sns.catplot('Heart Disease Present', col = 'Rest ECG Left Ventricular Hypertrophy', kind = 'count', data = heart)

In [ ]:
heart.groupby('Heart Disease Present').sum()[[column for column in heart.columns if 'Slope' in column]].plot.bar()

In [ ]:
heart.groupby('Heart Disease Present').sum()[[column for column in heart.columns if 'Thalasemmia' in column]].plot.bar()

In [ ]:
target_column = ['Heart Disease Present']
feature_columns = [column for column in heart.columns if column not in target_column]

x = heart[feature_columns].values
y = heart[target_column].values.ravel()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)

In [ ]:
x_train_scaled, x_test_scaled = map(scaler.transform, [x_train, x_test])

In [ ]:
kn = knn(weighted = 'weighted')

kn.fit(x_train_scaled, y_train)

y_pred = kn.predict(x_test_scaled)

print(accuracy_score(y_test, y_pred))

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k, weights = 'distance')
#knn = RadiusNeighborsClassifier()
knn.fit(x_train_scaled, y_train)

y_pred = knn.predict(x_test_scaled)

print(accuracy_score(y_test, y_pred))

In [ ]:
results = sm.Logit(y_train, x_train_scaled).fit()

In [ ]:
results.summary()

In [ ]:
results = sm.Logit(y, x).fit()
results.summary()

In [ ]:
y_pred = np.array([1 if y_ >= 0.5 else 0 for y_ in results.predict(x_test_scaled)])

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
#gb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
gb = XGBClassifier()
gb.fit(x_train_scaled, y_train)

In [ ]:
y_pred = gb.predict(x_test_scaled)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
target_column = ['Heart Disease Present']
feature_columns = [column for column in heart.columns if column not in target_column]

x = heart[feature_columns].values
y = heart[target_column].values.ravel()

print("Feature array shape:", x.shape)
print("Target array shape:", y.shape)

n_splits = 10
random_state = 5
kf = KFold(n_splits=n_splits, random_state=random_state)

print("K-Fold Accuracies for k = %s" % n_splits)

for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    
    x_train, x_test = scaler.transform(x_train), scaler.transform(x_test)
    
    model = KNeighborsClassifier(n_neighbors=5, weights='distance')
    #model = GradientBoostingClassifier(max_depth=5, n_estimators=100)
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    
    print(accuracy_score(y_test, y_pred))

In [ ]:
xx = np.random.randn(1000, 6)
yy = np.random.randint(2, size = 1000)

In [ ]:
kn = knn()

kn.fit(xx, yy)

sample = np.random.randn(6)

kn.predict(sample)

In [ ]:
class knn:
    
    def __init__(self, k = 5, weighted = 'uniform'):
        self.k = k
        self.weighted = weighted
        self.fitted = False
        
    def fit(self, features, targets):
        self.features = features
        self.targets = targets
        self.fitted = True
        
    def predict(self, x):
        
        if x.ndim == 1:
            x = x.reshape(1, -1)
        
        if self.weighted == 'uniform':
            f = self._uniform
        elif self.weighted == 'weighted':
            f = self._weighted
        else:
            raise ValueError("Weighted must be either 'uniform' or 'weighted'")
        
        pred_targets = np.zeros(len(x)).astype(int)
        for i, x_ in enumerate(x):
            
            pred_targets[i] = self._predict(x_, f)
            
        return pred_targets
    
    def _predict(self, x, f):
        
        distances = np.linalg.norm(self.features - x, axis = 1)
        
        kNearestInd = np.argsort(distances)[:self.k]
        
        return f(kNearestInd, distances)
        
    def _uniform(self, kNearestInd, distances):
        return self._mode(self.targets[kNearestInd])
    
    def _mode(self, y):
        uniqueValues, uniqueCounts = np.unique(y, return_counts = True)

        return uniqueValues[np.argmax(uniqueCounts)]
    
    def _weighted(self, kNearestInd, distances):
        
        valMap = {}

        for val, distance in zip(self.targets[kNearestInd], distances[kNearestInd]):
            if val not in valMap:
                valMap[val] = 0
            valMap[val] += (1 / distance if distance != 0 else 1 / 0.001)
            
        return max(valMap, key = valMap.get)
        
            
        
    

In [ ]:
k = 5

distances = np.linalg.norm(xx - sample, axis = 1)

def uniform(yy, kClosestInd):
    
    def mode(y):
        uniqueValues, uniqueCounts = np.unique(y, return_counts = True)

        return uniqueValues[np.argmax(uniqueCounts)]
    
    return mode(yy[kClosestInd])

def weighted(yy, distances, kClosestInd):
    
    valMap = {}

    for val, distance in zip(yy[kClosestInd], distances[kClosestInd]):
        if val not in valMap:
            valMap[val] = 0
        valMap[val] += 1 / distance
    
    return max(valMap, key = valMap.get)

kClosestInd = np.argsort(distances)[:k]

print(uniform(yy, kClosestInd))
print(weighted(yy, distances, kClosestInd))